In [4]:
import os
import sys
import glob
import pandas as pd
from metient.util import data_extraction_util as dutil
from metient.util import vertex_labeling_util as vutil
from metient.util import plotting_util as plutil

repo_dir = os.path.join(os.getcwd(), "../../")

def get_matching_basenames(directory, pattern, suffix):
    file_paths = glob.glob(os.path.join(directory, pattern))
    basenames = [os.path.basename(file_path).replace(suffix, "") for file_path in file_paths]
    return basenames

data_dir = os.path.join(repo_dir,'data/tracerx_nsclc/')
tsv_dir = os.path.join(data_dir,'patient_data/conipher_pyclone_clustered/')
patients = get_matching_basenames(tsv_dir, "*_SNVs.tsv", "_SNVs.tsv")
print(f"{len(patients)} patients")

sample_info_df= pd.read_csv(os.path.join(data_dir,"sample_overview_original.txt"), sep="\t")
sample_info_df


123 patients


patient_id tumour_id             region  sampleType sampleTypeDetail
0     CRUK0010  CRUK0010  CRUK0010_SU_T1.R1     primary          primary
1     CRUK0010  CRUK0010  CRUK0010_SU_T1.R2     primary          primary
2     CRUK0010  CRUK0010   CRUK0010_SU_FLN1  metastasis               LN
3     CRUK0010  CRUK0010    CRUK0010_BR_LN1  metastasis  metachronousMet
4     CRUK0010  CRUK0010    CRUK0010_BR_LN2  metastasis  metachronousMet
..         ...       ...                ...         ...              ...
689   CRUK0872  CRUK0872  CRUK0872_SU_T1.R1     primary          primary
690   CRUK0872  CRUK0872  CRUK0872_SU_T1.R2     primary          primary
691   CRUK0872  CRUK0872  CRUK0872_SU_T1.R3     primary          primary
692   CRUK0872  CRUK0872  CRUK0872_SU_T1.R4     primary          primary
693   CRUK0872  CRUK0872   CRUK0872_SU_FLN1  metastasis               LN

[694 rows x 5 columns]

### Replicate TRACERx's clone presence estimation

In [5]:
import numpy as np
import torch
    
def get_node_to_sample_to_ccf(cluster_df, samples, num_nodes):
    node_to_sample_to_phyloccf = {x:dict() for x in range(num_nodes)}
    for sample in samples:
        for node in range(num_nodes):
            row = cluster_df[(cluster_df['treeCLUSTER']==node)&(cluster_df['SAMPLE']==sample)]
            if row['clonality'].item() == 'clonal':
                mean_ccf = 1.0
            else:
                mean_ccf = float(row['meanCCF'].item())/100.0

            node_to_sample_to_phyloccf[node][sample] = mean_ccf
    return node_to_sample_to_phyloccf

def get_node_to_sample_to_clone_proportion(tree, samples, node_to_sample_to_phyloccf):
    node_to_sample_to_clone_proportion = {}
    leaves = vutil.get_leaves(tree)
    print("leaves", leaves)
    for leaf in leaves:
        node_to_sample_to_clone_proportion[leaf] = {}
        for sample in samples:
            node_to_sample_to_clone_proportion[leaf][sample] = node_to_sample_to_phyloccf[leaf][sample]
    
    
    reverse_bfs = vutil.reverse_bfs_order(tree)
    print("reverse_bfs", reverse_bfs) 
    for sample in samples:
        for node in reverse_bfs:
            if node not in node_to_sample_to_clone_proportion:
                node_to_sample_to_clone_proportion[node] = {}

            descendants = vutil.get_descendants(tree, node)
            descendant_ccf_sum = sum([node_to_sample_to_phyloccf[d][sample] for d in descendants])
            node_to_sample_to_clone_proportion[node][sample] = node_to_sample_to_phyloccf[node][sample]-descendant_ccf_sum
    
    
    return node_to_sample_to_clone_proportion

In [6]:
import csv
new_tsv_dir = os.path.join(data_dir,'patient_data/conipher_pyclone_clustered_phyloccf_presence/')

def process_patient(patient):
    print(patient)
    snvs_df = pd.read_csv(os.path.join(tsv_dir, f"{patient}_SNVs.tsv"), sep="\t", index_col=False)
    samples = snvs_df['sample_label'].unique()
    
    tree_fn = os.path.join(data_dir, 'conipher_outputs/TreeBuilding/', f"{patient}_conipher_SNVsallTrees_cleaned.txt")
    tree = dutil.get_adj_matrices_from_all_conipher_trees(tree_fn)[0]
    
    cluster_table_info_fn = os.path.join(data_dir, 'conipher_outputs/TreeBuilding/', f"{patient}_conipher_SNVsclusterInfo_cleaned.tsv")
    cluster_df = pd.read_csv(cluster_table_info_fn, sep="\t", index_col=False)

    num_nodes = tree.shape[0]
    node_to_sample_to_phyloccf = get_node_to_sample_to_ccf(cluster_df, samples, num_nodes)
    print(node_to_sample_to_phyloccf)
    df = pd.DataFrame(node_to_sample_to_phyloccf).T
    df.index.name = 'cluster'
    df.to_csv(os.path.join(data_dir,'patient_data/conipher_phyloccfs', f"{patient}_phyloCCFs.csv"))
    #print(df)
    
    node_to_sample_to_clone_proportion = get_node_to_sample_to_clone_proportion(tree, samples, node_to_sample_to_phyloccf)
    print(node_to_sample_to_clone_proportion)

    site_idx_to_samples = snvs_df.groupby('anatomical_site_index')['sample_label'].agg(set).to_dict()
    site_indices = snvs_df['anatomical_site_index'].unique()
    data = []
    for node in range(num_nodes):
        for site_idx in site_indices:
            associated_samples = site_idx_to_samples[site_idx]
            presence = 0
            for sample in associated_samples:
                if node_to_sample_to_clone_proportion[node][sample] > 0.0:
                    presence = 1
            subset = snvs_df[snvs_df['anatomical_site_index']==site_idx]
            site_label = subset['anatomical_site_label'].unique().item()
            site_category = subset['site_category'].unique().item()
            
            character_labels = snvs_df[snvs_df['cluster_index']==node]['character_label'].unique()
            character_label = plutil.get_pruned_mut_label(character_labels, True, True)
            num_mutations = len(character_labels)
            data.append([site_idx, site_label, node, character_label, presence, site_category, num_mutations])
    new_snvs_df = pd.DataFrame(data, columns=['anatomical_site_index', 'anatomical_site_label', 'cluster_index',
                                              'cluster_label', 'present', 'site_category', 'num_mutations'])
    
    output_fn = os.path.join(new_tsv_dir, f"{patient}_SNVs.tsv")
    with open(output_fn, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerow(new_snvs_df.columns)
        for _, row in new_snvs_df.iterrows():
            writer.writerow(row)
for patient in patients:
    process_patient(patient)
        

CRUK0721
{0: {'CRUK0721_SU_FLN1': 1.0, 'CRUK0721_SU_T1.R1': 1.0}, 1: {'CRUK0721_SU_FLN1': 0.6, 'CRUK0721_SU_T1.R1': 0.82}, 2: {'CRUK0721_SU_FLN1': 0.0, 'CRUK0721_SU_T1.R1': 0.58}, 3: {'CRUK0721_SU_FLN1': 0.0, 'CRUK0721_SU_T1.R1': 0.38}, 4: {'CRUK0721_SU_FLN1': 0.0, 'CRUK0721_SU_T1.R1': 0.22}, 5: {'CRUK0721_SU_FLN1': 0.27, 'CRUK0721_SU_T1.R1': 0.0}}
leaves [4, 5]
reverse_bfs [4, 3, 5, 2, 1, 0]
{4: {'CRUK0721_SU_FLN1': 0.0, 'CRUK0721_SU_T1.R1': 0.22}, 5: {'CRUK0721_SU_FLN1': 0.27, 'CRUK0721_SU_T1.R1': 0.0}, 3: {'CRUK0721_SU_FLN1': 0.0, 'CRUK0721_SU_T1.R1': 0.16}, 2: {'CRUK0721_SU_FLN1': 0.0, 'CRUK0721_SU_T1.R1': -0.020000000000000018}, 1: {'CRUK0721_SU_FLN1': 0.32999999999999996, 'CRUK0721_SU_T1.R1': -0.36}, 0: {'CRUK0721_SU_FLN1': 0.13, 'CRUK0721_SU_T1.R1': -0.9999999999999998}}
CRUK0325
{0: {'CRUK0325_BR_T1.FR1': 1.0, 'CRUK0325_SU_FLN1': 1.0, 'CRUK0325_SU_T1.R2': 1.0}, 1: {'CRUK0325_BR_T1.FR1': 0.71, 'CRUK0325_SU_FLN1': 0.56, 'CRUK0325_SU_T1.R2': 1.0}, 2: {'CRUK0325_BR_T1.FR1': 0.0, 'C

{0: {'CRUK0048_BR_T1.R1': 1.0, 'CRUK0048_BR_T1.R2': 1.0, 'CRUK0048_SU_T1.R1': 1.0, 'CRUK0048_SU_T1.R2': 1.0, 'CRUK0048_SU_T1.R3': 1.0}, 1: {'CRUK0048_BR_T1.R1': 1.0, 'CRUK0048_BR_T1.R2': 1.0, 'CRUK0048_SU_T1.R1': 0.27, 'CRUK0048_SU_T1.R2': 0.68, 'CRUK0048_SU_T1.R3': 0.76}, 2: {'CRUK0048_BR_T1.R1': 1.0, 'CRUK0048_BR_T1.R2': 1.0, 'CRUK0048_SU_T1.R1': 0.0, 'CRUK0048_SU_T1.R2': 0.0, 'CRUK0048_SU_T1.R3': 0.0}, 3: {'CRUK0048_BR_T1.R1': 0.0, 'CRUK0048_BR_T1.R2': 0.0, 'CRUK0048_SU_T1.R1': 0.55, 'CRUK0048_SU_T1.R2': 0.0, 'CRUK0048_SU_T1.R3': 0.0}, 4: {'CRUK0048_BR_T1.R1': 0.0, 'CRUK0048_BR_T1.R2': 0.0, 'CRUK0048_SU_T1.R1': 1.0, 'CRUK0048_SU_T1.R2': 0.0, 'CRUK0048_SU_T1.R3': 0.0}, 5: {'CRUK0048_BR_T1.R1': 0.0, 'CRUK0048_BR_T1.R2': 0.0, 'CRUK0048_SU_T1.R1': 0.3, 'CRUK0048_SU_T1.R2': 0.0, 'CRUK0048_SU_T1.R3': 0.0}, 6: {'CRUK0048_BR_T1.R1': 0.0, 'CRUK0048_BR_T1.R2': 0.0, 'CRUK0048_SU_T1.R1': 0.0, 'CRUK0048_SU_T1.R2': 0.5, 'CRUK0048_SU_T1.R3': 0.36}, 7: {'CRUK0048_BR_T1.R1': 0.22, 'CRUK0048_BR_T1.R2

CRUK0799
{0: {'CRUK0799_SU_FLN1': 0.0, 'CRUK0799_SU_FLN2': 1.0, 'CRUK0799_SU_T1.R1': 0.0, 'CRUK0799_SU_T1.R2': 0.0, 'CRUK0799_SU_T1.R3': 0.0}, 1: {'CRUK0799_SU_FLN1': 0.0, 'CRUK0799_SU_FLN2': 0.0, 'CRUK0799_SU_T1.R1': 0.0, 'CRUK0799_SU_T1.R2': 0.0, 'CRUK0799_SU_T1.R3': 1.0}, 2: {'CRUK0799_SU_FLN1': 0.0, 'CRUK0799_SU_FLN2': 0.0, 'CRUK0799_SU_T1.R1': 0.9, 'CRUK0799_SU_T1.R2': 0.0, 'CRUK0799_SU_T1.R3': 0.0}, 3: {'CRUK0799_SU_FLN1': 1.0, 'CRUK0799_SU_FLN2': 1.0, 'CRUK0799_SU_T1.R1': 1.0, 'CRUK0799_SU_T1.R2': 1.0, 'CRUK0799_SU_T1.R3': 1.0}, 4: {'CRUK0799_SU_FLN1': 1.0, 'CRUK0799_SU_FLN2': 0.0, 'CRUK0799_SU_T1.R1': 0.0, 'CRUK0799_SU_T1.R2': 1.0, 'CRUK0799_SU_T1.R3': 1.0}, 5: {'CRUK0799_SU_FLN1': 0.0, 'CRUK0799_SU_FLN2': 0.0, 'CRUK0799_SU_T1.R1': 0.0, 'CRUK0799_SU_T1.R2': 0.38, 'CRUK0799_SU_T1.R3': 0.0}, 6: {'CRUK0799_SU_FLN1': 0.0, 'CRUK0799_SU_FLN2': 0.0, 'CRUK0799_SU_T1.R1': 0.71, 'CRUK0799_SU_T1.R2': 0.0, 'CRUK0799_SU_T1.R3': 0.0}, 7: {'CRUK0799_SU_FLN1': 0.0, 'CRUK0799_SU_FLN2': 0.0, 'CR

CRUK0557
{0: {'CRUK0557_BR_T1.R1': 1.0, 'CRUK0557_SU_FLN1': 1.0, 'CRUK0557_SU_T1.R1': 1.0, 'CRUK0557_SU_T1.R2': 1.0, 'CRUK0557_SU_T1.R3': 1.0, 'CRUK0557_SU_T1.R4': 1.0, 'CRUK0557_SU_T1.R5': 1.0}, 1: {'CRUK0557_BR_T1.R1': 1.0, 'CRUK0557_SU_FLN1': 0.0, 'CRUK0557_SU_T1.R1': 0.0, 'CRUK0557_SU_T1.R2': 0.0, 'CRUK0557_SU_T1.R3': 0.0, 'CRUK0557_SU_T1.R4': 0.0, 'CRUK0557_SU_T1.R5': 0.0}, 2: {'CRUK0557_BR_T1.R1': 1.0, 'CRUK0557_SU_FLN1': 1.0, 'CRUK0557_SU_T1.R1': 0.0, 'CRUK0557_SU_T1.R2': 0.0, 'CRUK0557_SU_T1.R3': 0.0, 'CRUK0557_SU_T1.R4': 0.0, 'CRUK0557_SU_T1.R5': 1.0}, 3: {'CRUK0557_BR_T1.R1': 0.0, 'CRUK0557_SU_FLN1': 0.0, 'CRUK0557_SU_T1.R1': 0.0, 'CRUK0557_SU_T1.R2': 0.0, 'CRUK0557_SU_T1.R3': 0.0, 'CRUK0557_SU_T1.R4': 0.14, 'CRUK0557_SU_T1.R5': 0.0}, 4: {'CRUK0557_BR_T1.R1': 0.0, 'CRUK0557_SU_FLN1': 0.0, 'CRUK0557_SU_T1.R1': 0.0, 'CRUK0557_SU_T1.R2': 0.0, 'CRUK0557_SU_T1.R3': 0.0, 'CRUK0557_SU_T1.R4': 0.0, 'CRUK0557_SU_T1.R5': 0.43}, 5: {'CRUK0557_BR_T1.R1': 0.0, 'CRUK0557_SU_FLN1': 0.0, 'CR

CRUK0514
{0: {'CRUK0514_BR_T1.R1': 1.0, 'CRUK0514_SU_T1.R1': 1.0}, 1: {'CRUK0514_BR_T1.R1': 0.0, 'CRUK0514_SU_T1.R1': 1.0}, 2: {'CRUK0514_BR_T1.R1': 0.54, 'CRUK0514_SU_T1.R1': 0.0}, 3: {'CRUK0514_BR_T1.R1': 1.0, 'CRUK0514_SU_T1.R1': 0.0}, 4: {'CRUK0514_BR_T1.R1': 0.0, 'CRUK0514_SU_T1.R1': 0.21}, 5: {'CRUK0514_BR_T1.R1': 0.0, 'CRUK0514_SU_T1.R1': 0.5}}
leaves [2, 4]
reverse_bfs [4, 2, 5, 3, 1, 0]
{2: {'CRUK0514_BR_T1.R1': 0.54, 'CRUK0514_SU_T1.R1': 0.0}, 4: {'CRUK0514_BR_T1.R1': 0.0, 'CRUK0514_SU_T1.R1': 0.21}, 5: {'CRUK0514_BR_T1.R1': 0.0, 'CRUK0514_SU_T1.R1': 0.29000000000000004}, 3: {'CRUK0514_BR_T1.R1': 0.45999999999999996, 'CRUK0514_SU_T1.R1': 0.0}, 1: {'CRUK0514_BR_T1.R1': 0.0, 'CRUK0514_SU_T1.R1': 0.29000000000000004}, 0: {'CRUK0514_BR_T1.R1': -0.54, 'CRUK0514_SU_T1.R1': -0.71}}
CRUK0227
{0: {'CRUK0227_SU_FLN1': 1.0, 'CRUK0227_SU_T1.R2': 1.0}, 1: {'CRUK0227_SU_FLN1': 0.0, 'CRUK0227_SU_T1.R2': 0.68}, 2: {'CRUK0227_SU_FLN1': 0.0, 'CRUK0227_SU_T1.R2': 1.03}, 3: {'CRUK0227_SU_FLN1': 

...

{2: {'CRUK0510_SU_FLN1': 0.0, 'CRUK0510_SU_T1.R1': 0.0, 'CRUK0510_SU_T1.R2': 0.0, 'CRUK0510_SU_T1.R3': 0.15, 'CRUK0510_SU_T1.R4': 0.0, 'CRUK0510_SU_T1.R5': 0.0}, 4: {'CRUK0510_SU_FLN1': 0.0, 'CRUK0510_SU_T1.R1': 0.0, 'CRUK0510_SU_T1.R2': 0.0, 'CRUK0510_SU_T1.R3': 0.0, 'CRUK0510_SU_T1.R4': 0.0, 'CRUK0510_SU_T1.R5': 0.21}, 7: {'CRUK0510_SU_FLN1': 0.0, 'CRUK0510_SU_T1.R1': 0.0, 'CRUK0510_SU_T1.R2': 0.0, 'CRUK0510_SU_T1.R3': 0.0, 'CRUK0510_SU_T1.R4': 0.17, 'CRUK0510_SU_T1.R5': 0.0}, 10: {'CRUK0510_SU_FLN1': 0.0, 'CRUK0510_SU_T1.R1': 0.0, 'CRUK0510_SU_T1.R2': 0.18, 'CRUK0510_SU_T1.R3': 0.0, 'CRUK0510_SU_T1.R4': 0.0, 'CRUK0510_SU_T1.R5': 0.0}, 15: {'CRUK0510_SU_FLN1': 0.25, 'CRUK0510_SU_T1.R1': 0.0, 'CRUK0510_SU_T1.R2': 0.0, 'CRUK0510_SU_T1.R3': 0.0, 'CRUK0510_SU_T1.R4': 0.0, 'CRUK0510_SU_T1.R5': 0.0}, 17: {'CRUK0510_SU_FLN1': 0.0, 'CRUK0510_SU_T1.R1': 0.0, 'CRUK0510_SU_T1.R2': 0.45, 'CRUK0510_SU_T1.R3': 0.0, 'CRUK0510_SU_T1.R4': 0.0, 'CRUK0510_SU_T1.R5': 0.0}, 25: {'CRUK0510_SU_FLN1': 0.0, 

CRUK0010
{0: {'CRUK0010_BR_LN1': 1.0, 'CRUK0010_BR_LN2': 1.0, 'CRUK0010_BR_LN3': 1.0, 'CRUK0010_SU_FLN1': 1.0, 'CRUK0010_SU_T1.R1': 1.0, 'CRUK0010_SU_T1.R2': 1.0}, 1: {'CRUK0010_BR_LN1': 0.0, 'CRUK0010_BR_LN2': 0.0, 'CRUK0010_BR_LN3': 0.0, 'CRUK0010_SU_FLN1': 0.0, 'CRUK0010_SU_T1.R1': 1.0, 'CRUK0010_SU_T1.R2': 0.0}, 2: {'CRUK0010_BR_LN1': 0.0, 'CRUK0010_BR_LN2': 0.0, 'CRUK0010_BR_LN3': 0.0, 'CRUK0010_SU_FLN1': 0.0, 'CRUK0010_SU_T1.R1': 0.0, 'CRUK0010_SU_T1.R2': 0.19}, 3: {'CRUK0010_BR_LN1': 1.0, 'CRUK0010_BR_LN2': 1.0, 'CRUK0010_BR_LN3': 1.0, 'CRUK0010_SU_FLN1': 1.0, 'CRUK0010_SU_T1.R1': 1.0, 'CRUK0010_SU_T1.R2': 1.0}, 4: {'CRUK0010_BR_LN1': 0.35, 'CRUK0010_BR_LN2': 0.0, 'CRUK0010_BR_LN3': 0.0, 'CRUK0010_SU_FLN1': 0.0, 'CRUK0010_SU_T1.R1': 0.0, 'CRUK0010_SU_T1.R2': 0.0}, 5: {'CRUK0010_BR_LN1': 0.0, 'CRUK0010_BR_LN2': 0.0, 'CRUK0010_BR_LN3': 0.0, 'CRUK0010_SU_FLN1': 0.0, 'CRUK0010_SU_T1.R1': 0.0, 'CRUK0010_SU_T1.R2': 1.0}, 6: {'CRUK0010_BR_LN1': 0.0, 'CRUK0010_BR_LN2': 0.0, 'CRUK0010_BR

leaves [6, 7, 9, 16, 17, 20, 21]
reverse_bfs [9, 8, 20, 21, 7, 3, 14, 6, 10, 16, 18, 17, 15, 12, 5, 13, 11, 4, 19, 2, 1, 0]
{6: {'CRUK0693_SU_FLN1': 0.0, 'CRUK0693_SU_T1.R1': 0.0, 'CRUK0693_SU_T1.R2': 0.0, 'CRUK0693_SU_T1.R3': 0.0, 'CRUK0693_SU_T1.R4': 0.12}, 7: {'CRUK0693_SU_FLN1': 0.0, 'CRUK0693_SU_T1.R1': 0.0, 'CRUK0693_SU_T1.R2': 0.09, 'CRUK0693_SU_T1.R3': 0.0, 'CRUK0693_SU_T1.R4': 0.0}, 9: {'CRUK0693_SU_FLN1': 0.0, 'CRUK0693_SU_T1.R1': 0.1, 'CRUK0693_SU_T1.R2': 0.0, 'CRUK0693_SU_T1.R3': 0.0, 'CRUK0693_SU_T1.R4': 0.0}, 16: {'CRUK0693_SU_FLN1': 0.43, 'CRUK0693_SU_T1.R1': 0.0, 'CRUK0693_SU_T1.R2': 0.0, 'CRUK0693_SU_T1.R3': 0.0, 'CRUK0693_SU_T1.R4': 0.0}, 17: {'CRUK0693_SU_FLN1': 0.0, 'CRUK0693_SU_T1.R1': 0.0, 'CRUK0693_SU_T1.R2': 0.16, 'CRUK0693_SU_T1.R3': 0.0, 'CRUK0693_SU_T1.R4': 0.19}, 20: {'CRUK0693_SU_FLN1': 0.0, 'CRUK0693_SU_T1.R1': 0.0, 'CRUK0693_SU_T1.R2': 0.0, 'CRUK0693_SU_T1.R3': 0.07, 'CRUK0693_SU_T1.R4': 0.0}, 21: {'CRUK0693_SU_FLN1': 0.0, 'CRUK0693_SU_T1.R1': 0.0, 'CRUK0

{0: {'CRUK0794_SU_FLN1': 0.0, 'CRUK0794_SU_T1.R1': 0.0, 'CRUK0794_SU_T1.R2': 0.0, 'CRUK0794_SU_T1.R3': 0.0, 'CRUK0794_SU_T1.R4': 0.0, 'CRUK0794_SU_T1.R5': 0.44}, 2: {'CRUK0794_SU_FLN1': 1.0, 'CRUK0794_SU_T1.R1': 0.0, 'CRUK0794_SU_T1.R2': 0.0, 'CRUK0794_SU_T1.R3': 0.0, 'CRUK0794_SU_T1.R4': 0.0, 'CRUK0794_SU_T1.R5': 0.0}, 3: {'CRUK0794_SU_FLN1': 0.0, 'CRUK0794_SU_T1.R1': 0.0, 'CRUK0794_SU_T1.R2': 0.0, 'CRUK0794_SU_T1.R3': 0.0, 'CRUK0794_SU_T1.R4': 0.18, 'CRUK0794_SU_T1.R5': 0.0}, 6: {'CRUK0794_SU_FLN1': 0.0, 'CRUK0794_SU_T1.R1': 0.0, 'CRUK0794_SU_T1.R2': 0.12, 'CRUK0794_SU_T1.R3': 0.0, 'CRUK0794_SU_T1.R4': 0.0, 'CRUK0794_SU_T1.R5': 0.0}, 11: {'CRUK0794_SU_FLN1': 0.0, 'CRUK0794_SU_T1.R1': 0.17, 'CRUK0794_SU_T1.R2': 0.0, 'CRUK0794_SU_T1.R3': 0.0, 'CRUK0794_SU_T1.R4': 0.0, 'CRUK0794_SU_T1.R5': 0.0}, 12: {'CRUK0794_SU_FLN1': 0.0, 'CRUK0794_SU_T1.R1': 0.0, 'CRUK0794_SU_T1.R2': 0.0, 'CRUK0794_SU_T1.R3': 0.37, 'CRUK0794_SU_T1.R4': 0.0, 'CRUK0794_SU_T1.R5': 0.0}, 16: {'CRUK0794_SU_FLN1': 0.0, 'C

leaves [1, 9, 11, 15]
reverse_bfs [11, 15, 3, 1, 16, 8, 6, 9, 10, 7, 4, 12, 17, 14, 13, 5, 2, 0]
{1: {'CRUK0497_BR_T1.FR1': 0.0, 'CRUK0497_SU_T1.R1': 0.14, 'CRUK0497_SU_T1.R2': 0.0, 'CRUK0497_SU_T1.R3': 0.0}, 9: {'CRUK0497_BR_T1.FR1': 0.0, 'CRUK0497_SU_T1.R1': 0.0, 'CRUK0497_SU_T1.R2': 0.22, 'CRUK0497_SU_T1.R3': 0.0}, 11: {'CRUK0497_BR_T1.FR1': 0.0, 'CRUK0497_SU_T1.R1': 0.0, 'CRUK0497_SU_T1.R2': 0.0, 'CRUK0497_SU_T1.R3': 0.09}, 15: {'CRUK0497_BR_T1.FR1': 0.31, 'CRUK0497_SU_T1.R1': 0.0, 'CRUK0497_SU_T1.R2': 0.0, 'CRUK0497_SU_T1.R3': 0.0}, 3: {'CRUK0497_BR_T1.FR1': 0.0, 'CRUK0497_SU_T1.R1': 0.0, 'CRUK0497_SU_T1.R2': 0.0, 'CRUK0497_SU_T1.R3': 0.1}, 16: {'CRUK0497_BR_T1.FR1': 0.24000000000000005, 'CRUK0497_SU_T1.R1': 0.0, 'CRUK0497_SU_T1.R2': 0.0, 'CRUK0497_SU_T1.R3': 0.0}, 8: {'CRUK0497_BR_T1.FR1': 0.0, 'CRUK0497_SU_T1.R1': 0.0, 'CRUK0497_SU_T1.R2': 0.0, 'CRUK0497_SU_T1.R3': 0.009999999999999953}, 6: {'CRUK0497_BR_T1.FR1': 0.0, 'CRUK0497_SU_T1.R1': 0.20999999999999996, 'CRUK0497_SU_T1.R2'